In [1]:
import warnings
warnings.filterwarnings('ignore')
from numpy import set_printoptions
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
pd.options.display.max_columns=None
import seaborn as sns 
from pandas import read_csv
import io
import base64
import json
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage

import os
import os.path
from unidecode import unidecode

from urllib.error import HTTPError

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from mlens.ensemble import SuperLearner


from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import cohen_kappa_score

[MLENS] backend: threading


### CARGUE DE DATOS Y CONEXIÓN A DRIVE

In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/credentials.json', SCOPES) # Reemplazar con la ruta correcta
        creds = flow.run_local_server(port=0)
    with open('C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/token.json', 'w') as token:
        token.write(creds.to_json())
        
# Crear una instancia de la API de Drive
drive_service = build('drive', 'v3', credentials=creds)

# ID de la carpeta de Google Drive
folder_id = '1hQeetmO4XIObUefS_nzePqKqq3VksUEC'

# Ruta de destino para guardar los archivos descargados
save_path = 'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MODULO_ANALITICA_PRACTICO/DATOS'  # Reemplazar con la ruta deseada

# Función para descargar archivos de la carpeta de Drive
def download_folder(folder_id, save_path):
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed=false",
        fields='files(id, name)').execute()
    items = results.get('files', [])
    for item in items:
        file_id = item['id']
        file_name = item['name']
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(os.path.join(save_path, file_name), 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Descargando {file_name}: {int(status.progress() * 100)}%")
    print("Descarga completa")

# Descargar archivos de la carpeta
download_folder(folder_id, save_path)

# Listar archivos descargados
files = os.listdir(save_path)
print("Archivos descargados:")
for file in files:
    print(file)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=899973764197-biu188dkvsgi2al0fh29udm7keak0lh0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62379%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=3zNRKoqKERhfgjrNE5Kpu2zYdcN2br&access_type=offline
Descargando industrial9.csv: 100%
Descargando industrial4.csv: 100%
Descargando industrial3.csv: 100%
Descargando industrial10.csv: 100%
Descargando industrial6.csv: 100%
Descargando industrial5.csv: 100%
Descargando industrial2.csv: 100%
Descargando industrial8.csv: 100%
Descargando industrial7.csv: 100%
Descargando industrial1.csv: 100%
Descargando Limpieza de datos 1 UD.ipynb: 100%
Descargando sistemas10.csv: 100%
Descargando sistemas6.csv: 100%
Descargando sistemas2.csv: 100%
Descargando sistemas7.csv: 100%
Descargando sistemas4.csv: 100%
Descargando sistemas9.csv: 100%
Descargando sistemas5.csv: 100%
Descargando sistemas3.c

In [3]:
variables_por_carrera = { 
    'industrial': {
        '1': ['PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES','QUIMICA_ICFES','IDIOMA_ICFES','LOCALIDAD','RENDIMIENTO_UNO'],
        '2': ['LOCALIDAD_COLEGIO', 'PG_ICFES', 'CON_MAT_ICFES', 'FISICA_ICFES', 'BIOLOGIA_ICFES', 'IDIOMA_ICFES', 'LOCALIDAD', 'PROMEDIO_UNO', 'CAR_UNO', 'NCC_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_QUIMICA', 'NOTA_CFJC', 'NOTA_TEXTOS', 'NOTA_SEMINARIO', 'NOTA_EE_UNO','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_UNO', 'NAA_UNO', 'NOTA_DIFERENCIAL', 'NOTA_DIBUJO', 'NOTA_TEXTOS', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NCC_DOS', 'NCA_DOS', 'NAA_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_PBASICA', 'NOTA_EE_DOS', 'RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_UNO', 'NOTA_EE_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_ESTADISTICA_UNO', 'NOTA_TERMODINAMICA', 'NOTA_TGS', 'NOTA_EE_TRES','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_ALGEBRA', 'NOTA_INTEGRAL', 'NOTA_MATERIALES', 'NOTA_EE_DOS','PROMEDIO_TRES', 'NAA_TRES', 'NOTA_MULTIVARIADO', 'NOTA_TERMODINAMICA', 'NOTA_ECUACIONES', 'NOTA_ESTADISTICA_DOS', 'NOTA_FISICA_DOS', 'NOTA_MECANICA', 'NOTA_PROCESOSQ', 'RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_MATERIALES', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_ADMINISTRACION', 'NOTA_LENGUA_UNO', 'NOTA_EI_UNO', 'NOTA_EI_DOS', 'RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO', 'PROMEDIO_DOS', 'NOTA_EE_DOS', 'PROMEDIO_TRES', 'NOTA_MULTIVARIADO', 'NOTA_FISICA_DOS', 'PROMEDIO_CINCO', 'NOTA_PROCESOSM', 'NOTA_LENGUA_UNO', 'NOTA_EI_DOS', 'PROMEDIO_SEIS', 'NCA_SEIS', 'NOTA_PLINEAL', 'NOTA_DISENO', 'NOTA_EI_TRES','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'NOTA_LENGUA_UNO','PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_DOS', 'NOTA_EE_CUATRO', 'PROMEDIO_CINCO', 'PROMEDIO_SEIS', 'NOTA_IECONOMICA', 'PROMEDIO_SIETE', 'NAA_SIETE', 'NOTA_GRAFOS', 'NOTA_CALIDAD_UNO', 'NOTA_ERGONOMIA', 'NOTA_EI_CINCO', 'PROMEDIO_OCHO', 'NCC_OCHO', 'NOTA_LOG_UNO', 'NOTA_GOPERACIONES','NOTA_CALIDAD_DOS', 'NOTA_LENGUA_DOS', 'NOTA_CONTEXTO', 'RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS', 'PROMEDIO_SIETE', 'PROMEDIO_OCHO', 'NOTA_CALIDAD_DOS', 'PROMEDIO_NUEVE', 'NAA_NUEVE', 'NOTA_GRADO_UNO', 'NOTA_LOG_DOS', 'NOTA_FINANZAS', 'NOTA_HISTORIA', 'RENDIMIENTO_DIEZ']
    },
    'sistemas': {
        '1': ['CON_MAT_ICFES','IDIOMA_ICFES','LOCALIDAD_COLEGIO','BIOLOGIA_ICFES','QUIMICA_ICFES','PG_ICFES','LITERATURA_ICFES','RENDIMIENTO_UNO'],
        '2': ['PROMEDIO_UNO','NOTA_DIFERENCIAL','NOTA_CATEDRA_DEM','NOTA_PROG_BASICA','NAA_UNO','CAR_UNO','NOTA_TEXTOS','NOTA_LOGICA','NOTA_SEMINARIO','VECES_LOGICA','NAC_UNO','NCC_UNO','NCA_UNO','IDIOMA_ICFES','VECES_CATEDRA_DEM','NOTA_EE_UNO','PG_ICFES','NAP_UNO','VECES_DIFERENCIAL','CON_MAT_ICFES','NOTA_CATEDRA_CON','VECES_EE_UNO','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_UNO','PROMEDIO_DOS','NOTA_TGS','NOTA_PROG_AVANZADA','NOTA_FISICA_DOS','NOTA_MULTIVARIADO','NOTA_SEGUNDA_LENGUA_DOS','VECES_FISICA_DOS','NOTA_EE_DOS','VECES_PROG_AVANZADA','NAA_DOS','VECES_TGS','NCA_DOS','VECES_MULTIVARIADO','NOTA_ECUACIONES','NOTA_DIFERENCIAL','NAC_DOS','RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_UNO','PROMEDIO_DOS','NOTA_METODOS_NUM','NOTA_A_SISTEMAS','NOTA_MOD_PROG_UNO','NOTA_MAT_DIS','NOTA_HOMBRE','NOTA_GT_DOS','VECES_A_SISTEMAS','VECES_HOMBRE','NCC_TRES','NAA_DOS','NAA_TRES','NOTA_FUND_BASES','NAC_TRES','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_TRES','NOTA_PROB','NOTA_CIENCIAS_COMP_UNO','NOTA_MOD_PROG_DOS','NOTA_ARQ_COMP','NOTA_INV_OP_UNO','NOTA_FISICA_TRES','VECES_INV_OP_UNO','VECES_PROB','NOTA_METODOS_NUM','NOTA_MOD_PROG_UNO','VECES_MOD_PROG_DOS','VECES_FISICA_TRES','RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_TRES','NOTA_HISTORIA_COL','NOTA_ESTADISTICA','NOTA_CIBERNETICA_UNO','NOTA_INV_OP_DOS','NOTA_REDES_COM_UNO','NOTA_GI_UNO','VECES_HISTORIA_COL','VECES_CIBERNETICA_UNO','NOTA_CIENCIAS_COMP_DOS','VECES_REDES_COM_UNO','VECES_GI_UNO','VECES_ESTADISTICA','NOTA_MOD_PROG_DOS','VECES_CIENCIAS_COMP_DOS','VECES_INV_OP_DOS','CAR_CINCO','NOTA_METODOS_NUM','VECES_FISICA_TRES','RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_TRES','PROMEDIO_CINCO','NOTA_F_ING_SOF','NOTA_GT_TRES','NOTA_ECONOMIA','NOTA_EI_DOS','NOTA_REDES_COM_DOS','VECES_ECONOMIA','NOTA_CIBERNETICA_UDOS','NOTA_INV_OP_TRES','NOTA_INV_OP_DOS','NOTA_HISTORIA_COL','VECES_F_ING_SOF','VECES_REDES_COM_DOS','NOTA_METODOS_NUM','VECES_EI_DOS','VECES_CIBERNETICA_DOS','NOTA_ESTADISTICA','NCP_SEIS','NOTA_MOD_PROG_DOS','NOTA_GI_DOS','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_TRES','PROMEDIO_CINCO','PROMEDIO_SEIS','NOTA_DIS_ARQ_SOFT','NOTA_EI_OP_AI','NOTA_ING_ECONOMICA','NOTA_REDES_COM_TRES','NOTA_CIBERNETICA_TRES','VECES_EI_OP_AI','VECES_DIS_ARQ_SOFT','VECES_ING_ECONOMICA','VECES_EI_OP_BI','VECES_CIBERNETICA_TRES','NOTA_INV_OP_DOS','RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_TRES','PROMEDIO_CINCO','PROMEDIO_SEIS','PROMEDIO_SIETE','NOTA_EI_OP_AII','NOTA_GES_CAL_VAL_SOFT','NOTA_EI_OP_CI','NOTA_SIS_OP','VECES_EI_OP_AII','VECES_SIS_OP','NOTA_ING_ECONOMICA','VECES_EI_OP_CI','NAC_OCHO','VECES_GES_CAL_VAL_SOFT','RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_TRES','PROMEDIO_CINCO','PROMEDIO_SEIS','PROMEDIO_SIETE','NOTA_FOGEP','NOTA_EI_OP_DI','NOTA_EI_OP_CII','NOTA_EI_OP_CI','NOTA_EI_OP_AIII','NOTA_ING_ECONOMICA','VECES_FOGEP','NOTA_GRADO_UNO','NOTA_GRADO_DOS','VECES_EI_OP_DI','NOTA_EI_OP_BIII','NCP_NUEVE','NOTA_GES_CAL_VAL_SOFT','RENDIMIENTO_DIEZ']
    },
    'catastral': {
        '1': ['CON_MAT_ICFES','IDIOMA_ICFES','BIOLOGIA_ICFES','GENERO','FILOSOFIA_ICFES','LOCALIDAD','LITERATURA_ICFES','QUIMICA_ICFES','PG_ICFES','RENDIMIENTO_UNO'],
        '2': ['PROMEDIO_UNO','NOTA_ALGEBRA','NOTA_DITO','NOTA_TEXTOS','NOTA_CD&C','NCA_UNO','NAA_UNO','BIOLOGIA_ICFES','NOTA_HIST','QUIMICA_ICFES','NOTA_SEM','FILOSOFIA_ICFES','IDIOMA_ICFES','LITERATURA_ICFES','NOTA_DIF','CON_MAT_ICFES','NAP_UNO','VECES_HIST','VECES_ALGEBRA','NCP_UNO','GENERO','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_UNO','PROMEDIO_DOS','EPA_DOS','NOTA_TOP','NOTA_PROGB','NOTA_FIS_UNO','NOTA_HIST','LITERATURA_ICFES','NOTA_INT','VECES_TOP','NCA_UNO','IDIOMA_ICFES','NAP_UNO','VECES_HIST','NOTA_EYB','NOTA_DIF','RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_TRES','NOTA_DIF','NOTA_SUELOS','NOTA_POO','NOTA_FIS_DOS','NOTA_TOP','NOTA_EYB','IDIOMA_ICFES','NOTA_HIST','NOTA_ECUA','NOTA_PROGB','LITERATURA_ICFES','NCA_TRES','NOTA_FOTO','NOTA_CFJC','VECES_SUELOS','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_TRES','PROMEDIO_CUATRO','NOTA_GEOGEO','NOTA_TOP','NCA_CUATRO','NOTA_SUELOS','NOTA_POO','NOTA_PRII','NOTA_FOTO','EPA_CUATRO','LITERATURA_ICFES','NOTA_PROGB','NOTA_PROB','NOTA_ECUA','NOTA_CFJC','RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_TRES','PROMEDIO_CUATRO','PROMEDIO_CINCO','NOTA_SUELOS','NOTAS_SISC','NOTA_TOP','NOTA_GEOGEO','NOTA_POO','VECES_BD','NOTA_ME','NOTA_ECUA','CAR_CINCO','NOTA_PROGB','NOTA_FOTO','NOTA_CARTO','LITERATURA_ICFES','VECES_CARTO','NOTA_CFJC','RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_CUATRO','PROMEDIO_CINCO','PROMEDIO_SEIS','NOTA_GEO_FIS','NOTA_L_DOS','NOTA_ECUA','CAR_CINCO','NOTA_CARTO','NOTA_LEGIC','NOTA_GEOGEO','NOTA_POO','NOTA_CFJC','NOTA_INGECO','VECES_L_DOS','NOTA_SUELOS','NOTA_FOTO','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_DOS','PROMEDIO_CINCO','PROMEDIO_SEIS','PROMEDIO_SIETE','NOTA_L_DOS','NOTA_ECONOMET','NOTA_AVAP','CAR_CINCO','NOTA_POO','NOTA_ECUA','NOTA_FOTO','NOTA_GEOGEO','NOTA_EI_UNO','RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_CINCO','PROMEDIO_SEIS','PROMEDIO_SIETE','PROMEDIO_OCHO','NOTA_FOTO','NOTA_POO','NOTA_L_DOS','NOTA_FOTOD','NOTA_AVAM','NOTA_AVAP','NOTA_EI_CUATRO','NOTA_L_TRES','NOTA_EI_UNO','NOTA_ECONOMET','NOTA_FGEP','NOTA_ECUA','NOTA_GEOGEO','RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_CINCO','PROMEDIO_SEIS','PROMEDIO_SIETE','PROMEDIO_NUEVE','NOTA_TG_UNO','NOTA_L_DOS','NOTA_AVAP','NOTA_EI_CINCO','NOTA_ECONOMET','NOTA_AVAM','NOTA_FOTO','NOTA_FGEP','NOTA_EI_SEIS','NOTA_EE_DOS','NOTA_ECUA','RENDIMIENTO_DIEZ']
    },
    'electrica':{
        '1': ['IDIOMA_ICFES','PG_ICFES','FISICA_ICFES','ANO_INGRESO','APT_VERB_ICFES','QUIMICA_ICFES','FILOSOFIA_ICFES','GENERO','BIOLOGIA_ICFES','RENDIMIENTO_UNO'],
        '2': ['PROMEDIO_UNO','NOTA_POO','NOTA_INT','NOTA_FIS_DOS','NOTA_EYB','NOTA_DEM','NOTA_CIR_UNO','NOTA_HCI','VECES_INT','VECES_FIS_DOS','PG_ICFES','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_DOS','NOTA_ECUA','NOTA_TERMO','NOTA_CIR_DOS','NOTA_FIS_DOS','NOTA_EL_UNO','VECES_ECUA','NOTA_FIS_TRES','NOTA_MULTI','NCP_DOS','NCA_DOS','VECES_TERMO','RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_DOS','NOTA_EL_DOS','NOTA_MPI','NOTA_ME','NOTA_PYE','NOTA_ELD','VECES_ME','VECES_EL_DOS','NOTA_GEE','NOTA_CIR_TRES','VECES_PYE','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_CUATRO','NOTA_ASD','NOTA_CEM','NOTA_IO_UNO','NOTA_INSE','NOTA_IYM','NOTA_EE_UNO','NOTA_GENH','VECES_CEM','NOTA_EL_DOS','VECES_IYM','VECES_INSE','RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_CUATRO','NOTA_INGECO','NOTA_DDP','NOTA_CTRL','NOTA_CONEM','NOTA_TDE','VECES_INGECO','VECES_IO_DOS','VECES_DDP','NAC_CINCO','NOTAS_RDC','RENDIMIENTO_SEIS'],
        '7': ['VECES_ECO','NOTA_L_UNO','NOTA_ASP','NOTA_ELECPOT','NOTA_ECO','NOTA_MAQEL','VECES_ASP','NAC_CINCO','NOTA_AUTO','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_SIETE','NOTA_FGEP','NOTA_HIST','NOTA_CCON','NOTA_L_DOS','NOTA_AIELEC','NOTA_SUBEL','VECES_SUBEL','VECES_FGEP','VECES_L_DOS','VECES_HIST','NOTA_L_UNO','RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_SIETE','NOTA_L_TRES','NOTA_HSE','NOTA_ADMIN','NOTA_TGUNO','VECES_HSE','NOTA_EI_TRES','NOTA_EE_DOS','NOTA_PROTELEC','NCC_OCHO','VECES_L_TRES','VECES_PROTELEC','CAR_OCHO','VECES_ADMIN','RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_NUEVE','NOTA_EI_CINCO','NOTA_TGDOS','NOTA_EI_SEIS','NOTA_EI_TRES','NOTA_EE_TRES','NOTA_EE_DOS','NCC_NUEVE','VECES_EI_CINCO','NOTA_HSE','RENDIMIENTO_DIEZ']
    },
    'electronica':{
        '1': ['CON_MAT_ICFES','QUIMICA_ICFES','PG_ICFES','FISICA_ICFES','IDIOMA_ICFES','BIOLOGIA_ICFES','GENERO','LOCALIDAD','LITERATURA_ICFES','RENDIMIENTO_UNO'],
        '2': ['PROMEDIO_UNO','NOTA_DIF','CON_MAT_ICFES','EPA_UNO','FISICA_ICFES','PG_ICFES','NOTA_PROGB','NOTA_FIS_UNO','BIOLOGIA_ICFES','IDIOMA_ICFES','LITERATURA_ICFES','NCP_UNO','NOTA_TEXTOS','NOTA_CFJC','NAP_UNO','RENDIMIENTO_DOS'],
        '3': ['PROMEDIO_UNO','PROMEDIO_DOS','PG_ICFES','BIOLOGIA_ICFES','NAA_DOS','FISICA_ICFES','NOTA_INT','VECES_INT','NCA_DOS','NOTA_FIS_DOS','CON_MAT_ICFES','CAR_DOS','RENDIMIENTO_TRES'],
        '4': ['PROMEDIO_UNO','PROMEDIO_DOS','PROMEDIO_TRES','NOTA_INT','NCA_DOS','NCA_TRES','NOTA_FIS_DOS','PG_ICFES','NOTA_FIS_TRES','NOTA_EL_UNO','VECES_FIS_TRES','NOTA_DEM','RENDIMIENTO_CUATRO'],
        '5': ['PROMEDIO_TRES','PROMEDIO_CUATRO','NOTA_FIS_DOS','NOTA_EL_DOS','NOTA_CAMPOS','NOTA_FCD','VECES_FCD','NOTA_PROGAPL','NOTA_VARCOM','NCA_TRES','NOTA_INT','NCP_CUATRO','RENDIMIENTO_CINCO'],
        '6': ['PROMEDIO_TRES','PROMEDIO_CINCO','NOTA_EE_DOS','NCC_CINCO','NOTA_ADMICRO','NOTA_FCD','NAP_CINCO','NOTA_VARCOM','NOTA_TFM','VECES_ADMICRO','NOTA_EL_DOS','NAC_CINCO','RENDIMIENTO_SEIS'],
        '7': ['PROMEDIO_TRES','PROMEDIO_CINCO','PROMEDIO_SEIS','NOTA_TFM','NOTA_VARCOM','NOTA_COMANA','NOTA_MOYGE','NOTA_CCON','VECES_ADMICRO','VECES_COMANA','VECES_DDM','NOTAS_SYS','RENDIMIENTO_SIETE'],
        '8': ['PROMEDIO_TRES','PROMEDIO_SEIS','PROMEDIO_SIETE','NOTA_SISDIN','NOTA_ELECPOT','NOTA_COMDIG','NOTA_FDS','NOTA_VARCOM','NOTA_COMANA','NOTA_TFM','NCC_SIETE','RENDIMIENTO_OCHO'],
        '9': ['PROMEDIO_SEIS','PROMEDIO_OCHO','NOTA_STG','NOTA_C_UNO','NOTA_FDS','NOTA_ELECPOT','NOTA_INSIND','NOTA_SISDIN','NAP_OCHO','NOTA_INGECO','NOTA_TFM','RENDIMIENTO_NUEVE'],
        '10': ['PROMEDIO_SEIS','PROMEDIO_NUEVE','NOTA_INSIND','NOTA_TV','NOTA_C_UNO','NOTA_EI_IA','NOTA_ELECPOT','VECES_EI_IIA','NOTA_EI_IIIA','VECES_TV','RENDIMIENTO_DIEZ']
    }
}

In [4]:
def cargar_datos(carrera, semestre):
    
    ruta_archivo = f'C:/Users/Intevo/Desktop/UNIVERSIDAD DISTRITAL PROYECTO FOLDER/UNIVERSIDAD-DISTRITAL-PROYECTO/MODULO_ANALITICA_PRACTICO/DATOS/{carrera}{semestre}.csv'
    datos = pd.read_csv(ruta_archivo,sep=";")
    
    return datos
carrera="industrial"
semestre="2"


In [5]:
datos = cargar_datos(carrera, semestre)
columnas_filtradas = variables_por_carrera[carrera][semestre]
df = datos[columnas_filtradas]
print("DataFrame con columnas filtradas:")
df=df.astype(int)
df

DataFrame con columnas filtradas:


,LOCALIDAD_COLEGIO,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,BIOLOGIA_ICFES,IDIOMA_ICFES,LOCALIDAD,PROMEDIO_UNO,CAR_UNO,NCC_UNO,NAA_UNO,NOTA_DIFERENCIAL,NOTA_DIBUJO,NOTA_QUIMICA,NOTA_CFJC,NOTA_TEXTOS,NOTA_SEMINARIO,NOTA_EE_UNO,RENDIMIENTO_DOS
0,20,472,65,63,59,62,8,39,0,9,4,35,0,45,40,0,40,0,1
1,20,373,37,42,49,37,20,26,2,13,3,26,35,25,40,10,35,0,1
2,20,419,65,60,61,75,20,39,0,15,7,36,41,34,36,46,50,44,2
3,20,425,72,65,57,65,15,35,1,15,6,35,40,29,38,39,30,39,2
4,20,425,76,60,62,50,9,30,2,13,4,22,36,25,38,37,44,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,20,302,59,0,64,58,20,28,3,13,3,21,28,25,40,37,43,0,2
2635,20,415,63,80,61,47,8,24,0,13,3,14,22,22,32,39,42,0,0
2636,21,350,66,0,72,55,20,35,3,15,6,30,30,35,42,44,25,44,2
2637,20,436,68,72,60,69,11,34,1,13,5,34,43,26,43,31,42,0,2


In [6]:
def numero_a_letras(numero):
    numeros_letras = ['cero', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez']
    return numeros_letras[int(numero)]

semestre_en_letras = numero_a_letras(semestre)
print(semestre_en_letras)


dos


In [7]:
X = df.loc[:, ~df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]
Y = df.loc[:, df.columns.str.contains(f'RENDIMIENTO_{semestre_en_letras.upper()}')]                                                     
print("Separación de datos usando Pandas") 
print(X.shape, Y.shape)

Separación de datos usando Pandas
(2639, 18) (2639, 1)


In [8]:
Y = LabelEncoder().fit_transform(Y.astype('str'))                
print(X.shape, Y.shape)

(2639, 18) (2639,)


In [9]:
X_T_JOHNSON1 = X.copy(deep=True)
def transformacion_johnson(X):
    transformador_johnson = PowerTransformer(method='yeo-johnson', standardize=True).fit(X)
    datos_transformados = transformador_johnson.transform(X)
    set_printoptions(precision=3)
    print(datos_transformados[:5, :])
    datos_transformados_df = pd.DataFrame(data=datos_transformados, columns=X.columns)
    return datos_transformados_df
Xpandas_T_JOHNSON1 = transformacion_johnson(X_T_JOHNSON1)
Xpandas_T_JOHNSON1.head(2)

[[ 1.220e+00  1.211e+00 -6.616e-01  7.632e-01 -3.454e-01 -1.764e-03
  -6.872e-01  1.419e+00 -1.149e+00 -2.024e+00 -5.930e-01  9.884e-01
  -2.309e+00  2.011e+00  4.505e-01 -2.261e+00  4.796e-01 -1.137e+00]
 [ 1.220e+00 -7.496e-01 -3.639e+00  3.222e-01 -1.557e+00 -1.918e+00
   1.357e+00 -8.553e-01  8.286e-01 -8.445e-01 -1.059e+00 -8.645e-02
   2.954e-01 -3.371e-01  4.505e-01 -1.908e+00 -1.781e-01 -1.137e+00]
 [ 1.220e+00  1.221e-01 -6.616e-01  7.053e-01 -1.107e-01  1.041e+00
   1.357e+00  1.419e+00 -1.149e+00  9.052e-01  1.219e+00  1.115e+00
   1.063e+00  6.560e-01 -2.268e-01  1.187e+00  2.086e+00  9.408e-01]
 [ 1.220e+00  2.409e-01  1.055e-01  8.010e-01 -5.825e-01  2.365e-01
   5.649e-01  5.721e-01  1.494e-01  9.052e-01  5.492e-01  9.884e-01
   9.297e-01  9.014e-02  9.682e-02  4.205e-01 -7.408e-01  8.519e-01]
 [ 1.220e+00  2.409e-01  5.473e-01  7.053e-01  5.774e-03 -9.382e-01
  -4.939e-01 -2.987e-01  8.286e-01 -8.445e-01 -5.930e-01 -5.259e-01
   4.178e-01 -3.371e-01  9.682e-02  2.143e-0

,LOCALIDAD_COLEGIO,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,BIOLOGIA_ICFES,IDIOMA_ICFES,LOCALIDAD,PROMEDIO_UNO,CAR_UNO,NCC_UNO,NAA_UNO,NOTA_DIFERENCIAL,NOTA_DIBUJO,NOTA_QUIMICA,NOTA_CFJC,NOTA_TEXTOS,NOTA_SEMINARIO,NOTA_EE_UNO
0,1.219622,1.210515,-0.661598,0.763182,-0.345439,-0.001764,-0.687206,1.419232,-1.148708,-2.024134,-0.592955,0.988445,-2.309482,2.011394,0.450509,-2.260658,0.479615,-1.136816
1,1.219622,-0.749578,-3.638676,0.322178,-1.556639,-1.917870,1.356628,-0.855313,0.828565,-0.844541,-1.058731,-0.086453,0.295395,-0.337097,0.450509,-1.908173,-0.178081,-1.136816


In [10]:
X_trn, X_tst, Y_trn, Y_tst = train_test_split(Xpandas_T_JOHNSON1, Y, test_size=0.3, random_state=2)
print('DATOS: Son {} datos para entrenamiento y {} datos para prueba'.format(X_trn.shape[0], X_tst.shape[0]))

DATOS: Son 1847 datos para entrenamiento y 792 datos para prueba


### KNEIGHBORSCLASSIFIER

In [11]:
def entrenar_modelo_knn_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {
        'n_neighbors': [i for i in range(1, 18, 1)],
        'metric': ['euclidean', 'manhattan', 'minkowski'],
        'algorithm': ['auto', 'kd_tree','ball_tree','brute'],
        #'p': [i for i in range(1, 18,1)],
        'weights': ['uniform']
    }
    modelo = KNeighborsClassifier()
    semilla = 5
    num_folds = 10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_knn=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = KNeighborsClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo ,grid_resultado.best_params_
modelo_knn, mejores_hiperparametros_knn = entrenar_modelo_knn_con_transformacion(X_trn, Y_trn)
mejores_hiperparametros_knn

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 76.34253819036428
Mejor valor PARAMETRO usando k-fold: {'algorithm': 'auto', 'metric': 'manhattan', 'n_neighbors': 16, 'weights': 'uniform'}


{'algorithm': 'auto',
 'metric': 'manhattan',
 'n_neighbors': 16,
 'weights': 'uniform'}

In [12]:
Y_pred_prueba = modelo_knn.predict(X_tst)
accuracy_knn = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_knn

0.7777777777777778

### SVC

In [13]:
X_trn

,LOCALIDAD_COLEGIO,PG_ICFES,CON_MAT_ICFES,FISICA_ICFES,BIOLOGIA_ICFES,IDIOMA_ICFES,LOCALIDAD,PROMEDIO_UNO,CAR_UNO,NCC_UNO,NAA_UNO,NOTA_DIFERENCIAL,NOTA_DIBUJO,NOTA_QUIMICA,NOTA_CFJC,NOTA_TEXTOS,NOTA_SEMINARIO,NOTA_EE_UNO
1383,0.242446,-0.073235,0.769051,0.705255,-0.942771,-0.629148,-0.306378,-1.478424,0.828565,-0.844541,-0.592955,-1.824003,-0.171415,-1.835145,0.450509,0.013939,0.920486,-1.136816
1857,1.219622,-0.947695,0.215761,-1.398687,1.030662,0.717475,1.356628,-0.140536,0.149437,-0.844541,-0.055744,-1.133410,0.057376,0.200580,0.638978,1.655589,0.622694,-1.136816
856,-0.022967,0.914451,0.326126,1.051117,-0.227795,-0.629148,-0.306378,-2.066529,-1.148708,-0.844541,-1.754749,-2.036507,-2.249734,-1.764963,-2.198631,-1.684273,-0.740753,-1.136816
1268,0.371337,0.502120,1.102736,0.819744,-0.701961,0.798201,0.728627,0.199511,0.149437,0.905221,0.549199,-0.198701,1.198773,0.425613,0.096824,-0.275065,1.233860,0.813771
1487,-0.160059,-1.176591,0.658104,-1.398687,1.473673,-0.316946,-0.493923,-1.391086,-1.148708,-0.844541,-1.448248,-1.133410,-1.081375,-0.739465,-1.028698,0.214251,-2.138355,-1.136816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2514,1.219622,0.043593,0.547299,0.685626,-0.463672,-1.014924,-0.123898,-0.976196,-1.148708,-0.844541,-1.058731,-0.309390,0.669357,-0.541541,0.096824,-2.049246,0.479615,-1.136816
2347,1.219622,0.280719,-0.333777,0.625718,0.121723,-0.938184,1.356628,-0.140536,0.828565,-0.844541,-0.592955,-0.938379,1.476366,-0.541541,0.835383,1.655589,0.340398,-1.136816
1608,-0.748092,-0.858143,0.991369,-1.398687,1.030662,0.396158,-1.311311,0.025483,0.149437,-0.844541,-0.055744,1.241894,0.295395,-1.616401,0.638978,0.961289,0.073505,-1.136816
2541,1.219622,1.019489,-1.420583,0.801036,0.121723,-0.238423,-0.687206,0.381682,0.149437,0.905221,0.549199,0.027316,0.295395,1.377582,1.473593,0.013939,-0.178081,0.688686


In [14]:
def entrenar_modelo_svc_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'kernel':   ['rbf', 'poly', 'sigmoid','linear'], 
                'C': [i/10000 for i in range(8,12,1)],
                'max_iter':[i for i in range(1,3,1)],
                'gamma' : [i/100 for i in range(80,110,5)],
                'random_state':[i for i in range(1,5,1)],
                }
    modelo = SVC()
    semilla=5
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_svc=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = SVC(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo ,grid_resultado.best_params_
modelo_svc,mejores_hiperparametros_svc = entrenar_modelo_svc_con_transformacion(X_trn, Y_trn)
mejores_hiperparametros_svc

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 42.22326674500588
Mejor valor PARAMETRO usando k-fold: {'C': 0.0008, 'gamma': 0.8, 'kernel': 'sigmoid', 'max_iter': 2, 'random_state': 1}


{'C': 0.0008,
 'gamma': 0.8,
 'kernel': 'sigmoid',
 'max_iter': 2,
 'random_state': 1}

In [15]:
Y_pred_prueba = modelo_svc.predict(X_tst)
accuracy_svc = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_svc

0.4633838383838384

### DECISION TREE

In [16]:
def entrenar_modelo_tree_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {          
              'max_depth':[i for i in range(1,7,1)],
              'min_samples_split' :  [i for i in range(1,7,1)],  
              'min_samples_leaf' : [i for i in range(1,7,1)], 
              'max_features' : [i for i in range(1,7,1)], 
              'splitter': ["best", "random"],
              'random_state': [i for i in range(1,5,1)],
              #'criterion': ['gini','entropy','log_loss']
            
}
    modelo = DecisionTreeClassifier()
    semilla=5
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_tree=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = DecisionTreeClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

modelo_tree,mejores_hiperparametros_tree = entrenar_modelo_tree_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 77.20799059929494
Mejor valor PARAMETRO usando k-fold: {'max_depth': 6, 'max_features': 5, 'min_samples_leaf': 5, 'min_samples_split': 2, 'random_state': 4, 'splitter': 'best'}


In [17]:
Y_pred_prueba = modelo_tree.predict(X_tst)
accuracy_tree = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_tree

0.7790404040404041

### NAIVE BAYES

In [18]:
def entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = {}
    modelo = GaussianNB()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GaussianNB(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo
modelo_gaussian = entrenar_modelo_gaussian_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 66.91509988249119
Mejor valor PARAMETRO usando k-fold: {}


In [19]:
Y_pred_prueba = modelo_gaussian.predict(X_tst)
accuracy_gaussian = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_gaussian

0.6717171717171717

### LDA

In [20]:
def entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 'solver':  ['svd','lsqr','eigen'],
            'n_components':[1,2,3,4,5,6,7,8,9,10],
            'shrinkage': ['auto', 0.001, 0.01, 0.1, 0.5,1,10,100,1000],
            'tol':[i/1000 for i in range(1,100,1)]}
    modelo = LinearDiscriminantAnalysis()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_LDA=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LinearDiscriminantAnalysis(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo, grid_resultado.best_params_
modelo_LDA,mejores_hiperparametros_LDA = entrenar_modelo_LDA_con_transformacion(X_trn, Y_trn)



Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 74.33401880141011
Mejor valor PARAMETRO usando k-fold: {'n_components': 1, 'shrinkage': 0.1, 'solver': 'lsqr', 'tol': 0.001}


In [21]:
Y_pred_prueba = modelo_LDA.predict(X_tst)
accuracy_LDA = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_LDA

0.7588383838383839

### BAGGINGCLASSIFIER

In [22]:
def entrenar_modelo_BG_con_transformacion(X_trn, Y_trn,mejores_hiperparametros_tree):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'n_estimators': [i for i in range(750,760,5)],
            'max_samples' : [i/100.0 for i in range(70,90,5)],
            'max_features': [i/100.0 for i in range(75,85,5)],
            'bootstrap': [True], 
            'bootstrap_features': [True]}
    base_estimator= DecisionTreeClassifier(**mejores_hiperparametros_tree)
    semilla=7
    modelo = BaggingClassifier(estimator=base_estimator)
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_BG=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = BaggingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_
X_trn = X_trn
Y_trn = Y_trn 
modelo_BG,mejores_hiperparametros_BG = entrenar_modelo_BG_con_transformacion(X_trn, Y_trn,mejores_hiperparametros_tree)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 78.56022326674501
Mejor valor PARAMETRO usando k-fold: {'bootstrap': True, 'bootstrap_features': True, 'max_features': 0.75, 'max_samples': 0.85, 'n_estimators': 755}


In [23]:
Y_pred_prueba = modelo_BG.predict(X_tst)
accuracy_BG = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_BG

0.8106060606060606

### RANDOM FOREST

In [24]:
def entrenar_modelo_random_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'min_samples_split' : [1, 2 , 3,  4 , 6 , 8 , 10 , 15, 20 ],  
                'min_samples_leaf' : [ 1 , 3 , 5 , 7 , 9, 12, 15 ],
                'criterion':('gini','entropy','log_loss'),
                #'max_features':('sqrt','log2',None)
                #'class_weight':('balanced','balanced_subsample')
              }
    modelo = RandomForestClassifier()
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_random=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = RandomForestClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

modelo_random,mejores_hiperparametros_random = entrenar_modelo_random_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 80.0226204465335
Mejor valor PARAMETRO usando k-fold: {'criterion': 'entropy', 'min_samples_leaf': 5, 'min_samples_split': 8}


In [25]:
Y_pred_prueba = modelo_random.predict(X_tst)
accuracy_random = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_random

0.8207070707070707

### EXTRA TREES CLASSIFIER

In [26]:
def entrenar_modelo_extra_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'min_samples_split' : [i for i in range(1,10,1)], 
                'min_samples_leaf' : [i for i in range(0,10,1)],
                'min_samples_leaf':[i for i in range(0,10,1)],
                'min_samples_split':[i for i in range(0,10,1)],
                'criterion':('gini','entropy','log_loss')}
    semilla=7            
    modelo = ExtraTreesClassifier(random_state=semilla, 
                                n_estimators=40,
                                bootstrap=True) 
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_extra=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = ExtraTreesClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

modelo_extra,mejores_hiperparametros_extra = entrenar_modelo_extra_con_transformacion(X_trn, Y_trn)

Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 79.53407755581668
Mejor valor PARAMETRO usando k-fold: {'criterion': 'gini', 'min_samples_leaf': 2, 'min_samples_split': 5}


In [27]:
Y_pred_prueba = modelo_extra.predict(X_tst)
accuracy_extra = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_extra

0.8207070707070707

### ADABOOST

In [28]:
def entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'learning_rate' : [i/10000.0 for i in range(5,20,5)],
                  'n_estimators':[i for i in range(1,50,1)]}
    semilla=7            
    modelo = AdaBoostClassifier(estimator = None,  algorithm = 'SAMME.R', 
                                random_state= None) 
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_ADA=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = AdaBoostClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_
modelo_ADA, mejores_hiperparametros_ADA= entrenar_modelo_ADA_con_transformacion(X_trn, Y_trn)

KeyboardInterrupt: 

In [ ]:
Y_pred_prueba = modelo_ADA.predict(X_tst)
accuracy_ADA = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_ADA

0.6320132013201321

### GRADIENT BOOSTING

In [ ]:
def entrenar_modelo_GD_con_transformacion(X_trn, Y_trn):
    X_trn_transformado = X_trn
    parameters = { 
                'learning_rate' : [0.01, 0.05, 0.1,0.15],
                'n_estimators': [i for i in range(100,1200,100)],
                'loss':('log_loss','exponential'),
                'criterion':['friedman_mse']     
              }
    semilla=7
    modelo = GradientBoostingClassifier(random_state=semilla,
                                    n_estimators= 100,learning_rate= 0.1,max_depth= 2,
                                    min_samples_split= 2, min_samples_leaf= 3,max_features= 2)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_GD=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = GradientBoostingClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

X_trn = X_trn
Y_trn = Y_trn 

modelo_GD,mejores_hiperparametros_GD = entrenar_modelo_GD_con_transformacion(X_trn, Y_trn)

KeyboardInterrupt: 

In [ ]:
Y_pred_prueba = modelo_GD.predict(X_tst)
accuracy_GD = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_GD

0.6320132013201321

### XGB

In [ ]:
def entrenar_modelo_XB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'reg_alpha': [0,0.1,0.2,0.3,0.4,0.5],
                'reg_lambda':  [i/1000.0 for i in range(100,150,5)],
                'n_estimators':  [i for i in range(1,10,2)],
                'colsample_bytree': [0.1,0.3, 0.5,0.6,0.7,0.8, 0.9, 1,1.1],
                'objective' : ('binary:logistic', 'Multi: softprob'),
                'loss': ['log_loss'],
                'max_features':('sqrt','log2')
                }
    semilla=7
    modelo = XGBClassifier(random_state=semilla,subsample =1,max_depth =2)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_XB=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = XGBClassifier(**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

modelo_XB,mejores_hiperparametros_XB = entrenar_modelo_XB_con_transformacion(X_trn, Y_trn)

KeyboardInterrupt: 

In [ ]:
Y_pred_prueba = modelo_XB.predict(X_tst)
accuracy_XB = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_XB

0.8195050946142649

### CATBOOST

In [ ]:
def entrenar_modelo_CB_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {'border_count':[53],'l2_leaf_reg': [42],'learning_rate': [0.01],
                  'depth': [4, 6, 8],'thread_count': [4, 8, 12]
                  } 
    semilla=7
    modelo = CatBoostClassifier(random_state=semilla, verbose =0)
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_CB=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = CatBoostClassifier(verbose=0,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

modelo_CB,mejores_hiperparametros_CB = entrenar_modelo_CB_con_transformacion(X_trn, Y_trn)


Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 81.43749999999999
Mejor valor PARAMETRO usando k-fold: {'border_count': 53, 'depth': 6, 'l2_leaf_reg': 42, 'learning_rate': 0.01, 'thread_count': 4}


In [ ]:
Y_pred_prueba = modelo_CB.predict(X_tst)
accuracy_CB = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_CB

0.8136826783114993

### LIGHT GBM

In [ ]:
def entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn):
    # Aplicar la transformación Yeo-Johnson
    X_trn_transformado = X_trn
    parameters = {
    'min_child_samples' : [i for i in range(200, 10000, 100)],'colsample_bytree': [0.6, 0.8, 1.0],
    'boosting_type': ['gbdt', 'dart', 'goss'],'objective': ['binary', 'multiclass'],'random_state': [42]}
    semilla=7
    modelo = LGBMClassifier (random_state=semilla,                           
                            num_leaves =  10,max_depth = 1, n_estimators = 100,    
                            learning_rate = 0.1 ,class_weight=  None, subsample = 1,
                            colsample_bytree= 1, reg_alpha=  0, reg_lambda = 0,
                            min_split_gain = 0, boosting_type = 'gbdt')
    semilla=7
    num_folds=10
    kfold = StratifiedKFold(n_splits=num_folds, random_state=semilla, shuffle=True)
    metrica = 'accuracy'
    grid = GridSearchCV(estimator=modelo, param_grid=parameters, scoring=metrica, cv=kfold, n_jobs=-1)
    grid_resultado = grid.fit(X_trn, Y_trn)
    mejores_hiperparametros_LIGHT=grid_resultado.best_params_
    print("Resultados de GridSearchCV para el modelo")
    print("Mejor valor EXACTITUD usando k-fold:", grid_resultado.best_score_*100)
    print("Mejor valor PARAMETRO usando k-fold:", grid_resultado.best_params_)
    mejor_modelo = LGBMClassifier(verbose=-1,**grid_resultado.best_params_)
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    return mejor_modelo,grid_resultado.best_params_

modelo_LIGHT,mejores_hiperparametros_LIGHT = entrenar_modelo_LIGHT_con_transformacion(X_trn, Y_trn)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000403 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 462
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 13
[LightGBM] [Info] Start training from score -0.990880
[LightGBM] [Info] Start training from score -2.224467
[LightGBM] [Info] Start training from score -0.772461
[LightGBM] [Info] Start training from score -2.900422
[LightGBM] [Info] Start training from score -5.585999
Resultados de GridSearchCV para el modelo
Mejor valor EXACTITUD usando k-fold: 82.31249999999999
Mejor valor PARAMETRO usando k-fold: {'boosting_type': 'gbdt', 'colsample_bytree': 1.0, 'min_child_samples': 500, 'objective': 'multiclass', 'random_state': 42}


In [ ]:
Y_pred_prueba = modelo_LIGHT.predict(X_tst)
accuracy_LIGHT = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_LIGHT

0.8064046579330422

In [ ]:
mejores_hiperparametros_knn = modelo_knn.get_params()
mejores_hiperparametros_knn

mejores_hiperparametros_GD = modelo_GD.get_params()
mejores_hiperparametros_GD

mejores_hiperparametros_tree = modelo_tree.get_params()
mejores_hiperparametros_tree

mejores_hiperparametros_ADA = modelo_ADA.get_params()
mejores_hiperparametros_ADA

mejores_hiperparametros_extra = modelo_extra.get_params()
mejores_hiperparametros_extra

mejores_hiperparametros_random = modelo_random.get_params()
mejores_hiperparametros_random

mejores_hiperparametros_BG = modelo_BG.get_params()
mejores_hiperparametros_BG

mejores_hiperparametros_XB = modelo_XB.get_params()
mejores_hiperparametros_XB

{'objective': 'multi:softprob',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 1,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 9,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': 0,
 'reg_lambda': 0.1,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None,
 'loss': 'log_loss',
 'max_features': 'sqrt'}

### VOTACION DURA

In [ ]:
def entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG,
                                                   mejores_hiperparametros_XB):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(**mejores_hiperparametros_GD)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo7 = XGBClassifier(**mejores_hiperparametros_XB)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), 
                                    ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6),
                                    ('XGB',modelo7)],voting='hard') 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, 
                                    cv=kfold,scoring = metrica)
    mejores_hiperparametros_voting_hard=mejor_modelo.get_params
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo, mejores_hiperparametros_voting_hard
modelo_voting_hard, mejores_hiperparametros_voting_hard= entrenar_modelo_voting_hard_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG,
                                                   mejores_hiperparametros_XB)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 82.25  % 


In [ ]:
Y_pred_prueba = modelo_voting_hard.predict(X_tst)
accuracy_voting_hard = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_voting_hard

0.8136826783114993

### VOTACION SUAVE

In [ ]:
def entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = GradientBoostingClassifier(**mejores_hiperparametros_GD)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    metrica = 'accuracy'
    mejor_modelo = VotingClassifier(
    estimators=[('Gradient', modelo1), ('Adaboost', modelo2), ('Extratrees', modelo3),
                                    ('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)],
    voting='soft',weights=[0.9,0.7,0.9,0.9,0.9,0.9]) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_voting_soft=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_voting_soft

# Entrenar el modelo KNN con transformación Yeo-Johnson
modelo_voting_soft,mejores_hiperparametros_voting_soft = entrenar_modelo_voting_soft_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_GD,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 82.25  % 


In [ ]:
Y_pred_prueba = modelo_voting_soft.predict(X_tst)
accuracy_voting_soft = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_voting_soft

0.8151382823871907

### STACKING ( METAMODELO LINEAL)

In [ ]:
def entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    base_estimator=DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo2 = AdaBoostClassifier(**mejores_hiperparametros_ADA)
    modelo3 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo4 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo5 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo6 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimador_final = LogisticRegression()
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[ ('Adaboost', modelo2), ('Extratrees', modelo3),('Random Forest',modelo4),
                                    ('Bagging',modelo5),('Decision tree',modelo6)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_stacking_lineal=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_stacking_lineal

X_trn = X_trn
Y_trn = Y_trn 

modelo_stacking_lineal,mejores_hiperparametros_stacking_lineal = entrenar_modelo_stacking_lineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_ADA,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 81.74999999999999  % 


In [ ]:
Y_pred_prueba = modelo_stacking_lineal.predict(X_tst)
accuracy_stacking_lineal = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_stacking_lineal

0.8209606986899564

### STACKING (METAMODELO NO LINEAL)

In [ ]:
def entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG):
    X_trn_transformado = X_trn
    semilla= 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier (**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo3 = BaggingClassifier(**mejores_hiperparametros_BG)
    modelo4 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimador_final = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    metrica = 'accuracy'
    mejor_modelo = StackingClassifier(
    estimators=[  ('Extratrees', modelo1),('Random Forest',modelo2),('Bagging',modelo3),
                                    ('Decision tree',modelo4)], 
                                    final_estimator=estimador_final) 
    mejor_modelo.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_stacking_nolineal=mejor_modelo.get_params
    resultados = cross_val_score(mejor_modelo, X_trn_transformado, Y_trn, cv=kfold,scoring = metrica)
    print("Rendimiento del modelo:") 
    print ("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean()*100," % ") 
    return mejor_modelo,mejores_hiperparametros_stacking_nolineal
X_trn = X_trn
Y_trn = Y_trn 
modelo_stacking_nolineal,mejores_hiperparametros_stacking_nolineal = entrenar_modelo_stacking_nolineal_con_transformacion(X_trn, Y_trn,
                                                   mejores_hiperparametros_tree,
                                                   mejores_hiperparametros_extra,
                                                   mejores_hiperparametros_random,
                                                   mejores_hiperparametros_BG)

Rendimiento del modelo:
Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold: 81.5625  % 


In [ ]:
Y_pred_prueba = modelo_stacking_nolineal.predict(X_tst)
accuracy_stacking_nolineal = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_stacking_nolineal

0.8282387190684134

### VOTING WEIGHT

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

def weighted_voting_ensemble(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_ADA,
                            mejores_hiperparametros_extra,mejores_hiperparametros_random,
                            mejores_hiperparametros_BG,mejores_hiperparametros_XB):
    X_trn_transformado = X_trn
    modelos = [
        GradientBoostingClassifier(**mejores_hiperparametros_GD),
        AdaBoostClassifier(**mejores_hiperparametros_ADA),
        ExtraTreesClassifier(**mejores_hiperparametros_extra),
        RandomForestClassifier(**mejores_hiperparametros_random),
        BaggingClassifier(**mejores_hiperparametros_BG),
        DecisionTreeClassifier(**mejores_hiperparametros_tree),
        XGBClassifier(**mejores_hiperparametros_XB)
    ]
    pesos = np.ones(len(modelos))
    precisión_modelos = []
    for modelo in modelos:
        modelo.fit(X_trn_transformado, Y_trn)
        precisión = np.mean(cross_val_score(modelo, X_trn_transformado, Y_trn, cv=KFold(n_splits=10, random_state=7, shuffle=True), scoring='accuracy'))
        precisión_modelos.append(precisión)
    suma_precisión = sum(precisión_modelos)
    for i in range(len(precisión_modelos)):
        pesos[i] = suma_precisión / (precisión_modelos[i] * len(modelos))

    return modelos, pesos

def weighted_voting_predict(modelos, pesos, X_test):
    predicciones_modelos = [modelo.predict(X_test) for modelo in modelos]
    pesos_float64 = np.array(pesos, dtype=np.float64)
    predicciones_ponderadas = np.zeros_like(predicciones_modelos[0], dtype=np.float64)
    for i, predicciones_modelo in enumerate(predicciones_modelos):
        predicciones_ponderadas += predicciones_modelo * pesos_float64[i]
    predicciones_ponderadas /= len(modelos)
    predicciones_ponderadas = np.round(predicciones_ponderadas).astype(int)
    
    return predicciones_ponderadas

In [ ]:
modelos, pesos = weighted_voting_ensemble(X_trn, Y_trn, mejores_hiperparametros_GD,
                            mejores_hiperparametros_tree,mejores_hiperparametros_ADA,
                            mejores_hiperparametros_extra,mejores_hiperparametros_random,
                            mejores_hiperparametros_BG,mejores_hiperparametros_XB)
Y_pred_prueba= weighted_voting_predict(modelos, pesos, X_tst)
accuracy_voting_weighted = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_voting_weighted

0.8136826783114993

### SUPER APRENDIZ

In [31]:
def entrenar_modelo_super_aprendiz(X_trn, Y_trn, 
                            mejores_hiperparametros_tree,mejores_hiperparametros_extra,
                            mejores_hiperparametros_random):
    X_trn_transformado = X_trn
    semilla = 5 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier(**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    modelo4 = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimadores = [('Extratrees', modelo1), ('Random Forest', modelo2),('DecisionTree',modelo4)
                ]
    super_learner = SuperLearner(folds=10, random_state=semilla, verbose=2)
    super_learner.add(estimadores)
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                        bootstrap=False, max_depth=11, min_samples_split=4, 
                                        min_samples_leaf=1)
    super_learner.add_meta(estimador_final)
    super_learner.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_super_learner=super_learner.get_params
    resultados = cross_val_score(super_learner, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return super_learner,mejores_hiperparametros_super_learner,estimadores
modelo_superaprendiz,mejores_hiperparametros_super_learner,estimadores = entrenar_modelo_super_aprendiz(X_trn, Y_trn,
                            mejores_hiperparametros_tree,mejores_hiperparametros_extra,
                            mejores_hiperparametros_random)


Fitting 2 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:02

Fitting 2 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:02

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:02

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 2 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:02

Predicting 2 layers
Processing layer-1 

In [33]:
Y_pred_prueba = modelo_superaprendiz.predict(X_tst)
accuracy_superaprendiz = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_superaprendiz


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00


0.8156565656565656

### SUPER APRENDIZ DOS CAPAS

In [ ]:
Y_trn = Y_trn.astype(int)
def entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn, mejores_hiperparametros_tree,
                                            mejores_hiperparametros_extra,mejores_hiperparametros_random
                                            ):
    X_trn_transformado = X_trn
    semilla = 7 
    kfold = StratifiedKFold(n_splits=10, random_state=semilla, shuffle=True)
    modelo1 = ExtraTreesClassifier(**mejores_hiperparametros_extra)
    modelo2 = RandomForestClassifier(**mejores_hiperparametros_random)
    model = DecisionTreeClassifier(**mejores_hiperparametros_tree)
    estimadores = [('Extratrees', modelo1), ('Random Forest', modelo2)
                 ]
    superaprendiz_dos_capas = SuperLearner(folds=10, random_state=semilla, verbose=2)
    superaprendiz_dos_capas.add(estimadores)
    superaprendiz_dos_capas.add(estimadores)
    estimador_final = ExtraTreesClassifier(n_estimators=100, max_features=None,
                                        bootstrap=False, max_depth=11, min_samples_split=4, 
                                        min_samples_leaf=1)
    superaprendiz_dos_capas.add_meta(estimador_final)
    
    superaprendiz_dos_capas.fit(X_trn_transformado, Y_trn)
    mejores_hiperparametros_superaprendiz_dos_capas=superaprendiz_dos_capas.get_params
    resultados = cross_val_score(superaprendiz_dos_capas, X_trn_transformado, Y_trn, cv=kfold, scoring='accuracy')
    print("Rendimiento del modelo:") 
    print("Promedio de Exactitud (DATOS ENTRENAMIENTO) usando k-fold:", resultados.mean() * 100, "%") 
    return superaprendiz_dos_capas,mejores_hiperparametros_superaprendiz_dos_capas,estimadores
modelo_superaprendiz_dos_capas,mejores_hiperparametros_superaprendiz_dos_capas,estimadores = entrenar_modelo_super_aprendiz_dos_capas(X_trn, Y_trn, mejores_hiperparametros_tree,
                                            mejores_hiperparametros_extra,mejores_hiperparametros_random)


Fitting 3 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:01
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:04

Fitting 3 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:01
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:03

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00

Fitting 3 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:01
Processing layer-3             done | 00:00:00
Fit complete                        | 00:00:04

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict com

In [ ]:
Y_pred_prueba = modelo_superaprendiz_dos_capas.predict(X_tst)
accuracy_superaprendiz_dos_capas = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_superaprendiz_dos_capas


Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:00


0.8136826783114993

### RED NEURONAL

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
np.random.seed(42)

torch.manual_seed(42)
torch.cuda.manual_seed(42)
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train_neural_network(X_train, y_train, input_size, hidden_size, num_classes, num_epochs=100, learning_rate=0.0015):
    # Convertir datos de entrenamiento a tensores de PyTorch
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    # Inicializar modelo y función de pérdida
    model = NeuralNetwork(input_size, hidden_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # Entrenamiento del modelo
    for epoch in range(num_epochs):
        # Forward pass y pérdida
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    return model,num_epochs,learning_rate

def evaluate_model(model, X_test, y_test):
    # Convertir datos de prueba a tensores de PyTorch
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # Obtener las predicciones del modelo
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

    # Calcular las métricas del modelo
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted.numpy())
    precision = precision_score(y_test, predicted.numpy(), average='weighted')
    recall = recall_score(y_test, predicted.numpy(), average='weighted')
    f1 = f1_score(y_test, predicted.numpy(), average='weighted')

    print('Accuracy: {:.2f}%'.format(accuracy * 100))
    print("Precisión: ", precision)
    print("Exhaustividad (Recall): ", recall)
    print("Puntuación F1: ", f1)

X_train_array = X_trn.values
X_test_array = X_tst.values
Y_trn = Y_trn 
input_size = X_trn.shape[1]
num_classes = 5
hidden_size =155

# Supongamos que X_train, y_train, X_test, y_test son tus datos de entrenamiento y prueba
modelo_red_reuronal,num_epochs,learning_rate = train_neural_network(X_train_array , Y_trn, input_size, hidden_size, num_classes)
evaluate_model(modelo_red_reuronal, X_test_array , Y_tst)

Epoch [10/100], Loss: 1.1126
Epoch [20/100], Loss: 0.7597
Epoch [30/100], Loss: 0.6702
Epoch [40/100], Loss: 0.6291
Epoch [50/100], Loss: 0.6114
Epoch [60/100], Loss: 0.5959
Epoch [70/100], Loss: 0.5841
Epoch [80/100], Loss: 0.5742
Epoch [90/100], Loss: 0.5657
Epoch [100/100], Loss: 0.5582
Accuracy: 80.20%
Precisión:  0.7195437620559635
Exhaustividad (Recall):  0.8020378457059679
Puntuación F1:  0.74558032665812


In [ ]:
X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)
outputs = modelo_red_reuronal(X_test_tensor)
_, predicted = torch.max(outputs, 1)
Y_pred_prueba = predicted.numpy()
accuracy_red_neuronal = accuracy_score(Y_tst, Y_pred_prueba)
accuracy_red_neuronal

0.8020378457059679

### COMPARACIÓN DE MÉTRICAS

In [ ]:
exactitudes = {
    'KNEIGHBORSCLASSIFIER': accuracy_knn,
    'SVC': accuracy_svc,
    'DECISION TREE': accuracy_tree,
    'NAIVE BAYES':accuracy_gaussian,
    'LDA':accuracy_LDA,
    'BAGGING':accuracy_BG,
    'RANDOM FOREST':accuracy_random,
    'EXTRATREES':accuracy_extra,
    'ADABOOST':accuracy_ADA,
    'GRADIENTBOOST': accuracy_GD,
    'XGBOOST':accuracy_XB,
    'CATBOOST':accuracy_CB,
    'LIGHT':accuracy_LIGHT,
    'VOTING HARD':accuracy_voting_hard,
    'VOTING SOFT':accuracy_voting_soft,
    'STACKING LINEAL':accuracy_stacking_lineal,
    'STACKING NO LINEAL':accuracy_stacking_nolineal,
    'VOTING WEIGHTED':accuracy_voting_weighted,
    'SUPER APRENDIZ':accuracy_superaprendiz,
    'SUPER APRENDIZ DOS CAPAS':accuracy_superaprendiz_dos_capas,
    'RED NEURONAL':accuracy_red_neuronal
}
mejor_modelo = max(exactitudes, key=exactitudes.get)
mejor_accuracy = exactitudes[mejor_modelo]

print("Mejor modelo:", mejor_modelo)
print("Exactitud prueba:", mejor_accuracy)


Mejor modelo: STACKING NO LINEAL
R^2 en prueba: 0.8282387190684134
